<a href="https://colab.research.google.com/github/Nkusi-Richard/COLAB-PROJECTS/blob/main/GPT_FROM_SCRATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import time
import numpy as np
import torch.nn.functional as F
import pickle

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 64
batch_size = 128

max_iters = 10000
eval_iters = 250
n_embed = 384
n_layer = 4
dropout = 0.2
n_head = 4

cuda


In [ ]:
with open ('/content/wizard_of_oz.txt', 'r', encoding= 'utf-8') as f:
  text = f.read()

print(text[:200])

chars = sorted(set(text))
print(chars)

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW 
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Encoder and Decoder. Encoder assigns int to chars, Decoder assings chars to int

In [ ]:
string_to_int = { char:i for i, char in enumerate(chars)}

int_to_string = { i:char for i, char in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

Splitting the data

In [ ]:
# training size is 80% of the whole data

n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]

x = train_data[:block_size]
y = train_data[1:block_size+1]

some torch examples

In [ ]:
randint = torch.randint(-100, 100, (6,))
print(randint)

tensor = torch.tensor([[1,2,3], [4,5,6]])
print(tensor)

zeros = torch.zeros((3,2))
print(zeros)

ones = torch.ones((3,2))
print(ones)

input = torch.empty(2,3)
input

arange = torch.arange(5)
print(arange)

linspace = torch.linspace(3,10, steps=5)
linspace

logspace = torch.logspace(start=-10, end=10, steps=5)
logspace

eye = torch.eye(5)
print(eye)

start_time = time.time()
zeros = torch.zeros(2,2)
end_time = time.time()

elapsed_time = end_time - start_time

torch_rand_gpu = torch.rand(100, 100, 100, 100).to(device)
torch_rand_gpu2 = torch.rand(100, 100, 100, 100).to(device)

torch_rand_cpu = torch.rand(100, 100, 100, 100)
torch_rand_cpu2 = torch.rand(100, 100, 100, 100)

start_time = time.time()

gpu_multiplication = torch_rand_gpu @ torch_rand_gpu2

end_time = time.time()

print(end_time - start_time)

start_time = time.time()
cpu_multiplication = np.multiply(torch_rand_cpu , torch_rand_cpu2)

end_time = time.time()
print(end_time - start_time)


tensor([-48,  12,  98, -61, -48,  79])
tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([0, 1, 2, 3, 4])
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
0.11809587478637695
0.1564197540283203


In [ ]:
probabilities = torch.tensor([0.5, 0.5])

samples = torch.multinomial(probabilities, num_samples=1, replacement=True)

print(samples)

tensor([1])


In [ ]:
tensor = torch.tensor([1, 2, 3, 4])
out = torch.cat((tensor, torch.tensor([5])), dim=0)
print(out)

tensor([1, 2, 3, 4, 5])


In [ ]:
out = torch.tril(torch.ones(5, 5))
print(out)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


In [ ]:
out = torch.triu(torch.ones(5, 5))
print(out)

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])


In [ ]:
out = torch.zeros(5, 5).masked_fill(torch.tril(torch.ones(5,5)) == 0, float('-inf') )
print(out)

print(torch.exp(out))
input = torch.zeros(2, 3, 4)
print(input.transpose(0, 1).shape)

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])
tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])
torch.Size([3, 2, 4])


In [ ]:
tensor1 = torch.tensor([1, 2, 3])
tensor2 = torch.tensor([1, 3, 2])
tensor3 = torch.tensor([3, 2, 1])
stack = torch.stack([tensor1, tensor2, tensor3])
print(stack)

tensor([[1, 2, 3],
        [1, 3, 2],
        [3, 2, 1]])


In [ ]:
import torch.nn as nn

input = torch.tensor([1, 2, 3, 4], dtype=torch.float)

linear_layer = nn.Linear(4, 3, bias=False)
print(linear_layer(input))

print(torch.softmax(linear_layer(input), dim=0))


print(F.softmax(linear_layer(input), dim=0))

tensor([1.9212, 1.5074, 0.6583], grad_fn=<SqueezeBackward4>)
tensor([0.5144, 0.3401, 0.1455], grad_fn=<SoftmaxBackward0>)
tensor([0.5144, 0.3401, 0.1455], grad_fn=<SoftmaxBackward0>)


Embeddings

In [ ]:
vocab_size = len(chars)
embedding_dim = 6
embedding_layer = nn.Embedding(vocab_size, embedding_dim)
indices = torch.tensor([1, 2, 3, 4], dtype=torch.long)
emb = embedding_layer(indices)
print(emb)

tensor([[ 2.4692, -0.6631,  1.8234, -1.2775, -0.1305, -0.5238],
        [-0.2195, -0.9336,  0.4795,  0.0438,  0.1355,  0.9338],
        [-1.0994, -1.0881, -0.2712,  0.5439, -1.9703, -0.8633],
        [ 1.8264, -0.6988, -1.4301, -0.6478,  2.2244, -0.4473]],
       grad_fn=<EmbeddingBackward0>)


Get Batch

In [ ]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  idx = torch.randint(len(data) - block_size, (batch_size,))

  x = torch.stack([data[i:i+block_size ] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx ])

  x, y = x.to(device), y.to(device)

  return x, y

x, y = get_batch('train')

print(x)
print(y)

tensor([[69, 58, 68,  ...,  1, 76, 62],
        [61, 58, 71,  ..., 72,  1, 55],
        [62, 71,  1,  ...,  1, 55, 74],
        ...,
        [68, 58, 72,  ..., 23,  1, 56],
        [72, 54, 62,  ...,  9,  1, 33],
        [ 1, 69, 65,  ..., 58, 57,  1]], device='cuda:0')
tensor([[58, 68, 69,  ..., 76, 62, 65],
        [58, 71,  1,  ...,  1, 55, 58],
        [71,  1, 61,  ..., 55, 74, 73],
        ...,
        [58, 72,  1,  ...,  1, 56, 54],
        [54, 62, 57,  ...,  1, 33,  0],
        [69, 65, 54,  ..., 57,  1, 65]], device='cuda:0')


Bigram Model

In [ ]:
class BigramModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()

    self.look_up_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets=None):

    logits = self.look_up_table(index)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for token in range(max_new_tokens):
      logits, loss = self.forward(index)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)

      next_index = torch.multinomial(probs, num_samples=1, replacement=True)

      index = torch.cat((index, next_index), dim=-1)

    return index

model = BigramModel(len(chars))
m = model.to(device)

#index = torch.tensor([1, 2, 3, 4])

#logits = model(index, targets=index)
#print(logits.shape)

Estimating Losses

In [ ]:
@torch.no_grad()

def estimates_losses():
  splits = ['train', 'val']
  out = {}
  m.eval()
  for split in splits:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      x, y = get_batch(split)
      logits, loss = m.forward(x, y)
      losses[k] = loss.item()
    out[split] = losses.mean()

  m.train()
  return out


Training Loop

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=3e-4)

for iter in range(max_iters):

  if iter % eval_iters == 0:
    losses = estimates_losses()
    print(losses)

  x, y = get_batch('train')
  logits, loss = m.forward(x, y)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

with open('model_001.pkl', 'wb') as f:
  pickle.dump(m, f)
print("model saved!")

{'train': tensor(8.6675), 'val': tensor(8.7149)}
{'train': tensor(2.7803), 'val': tensor(2.8363)}
{'train': tensor(2.5762), 'val': tensor(2.6295)}
{'train': tensor(2.5183), 'val': tensor(2.5710)}
{'train': tensor(2.4929), 'val': tensor(2.5484)}
{'train': tensor(2.4818), 'val': tensor(2.5354)}
{'train': tensor(2.4722), 'val': tensor(2.5258)}
{'train': tensor(2.4667), 'val': tensor(2.5126)}
{'train': tensor(2.4614), 'val': tensor(2.5073)}
{'train': tensor(2.4496), 'val': tensor(2.5068)}
{'train': tensor(2.4427), 'val': tensor(2.4969)}
{'train': tensor(2.4320), 'val': tensor(2.4971)}
{'train': tensor(2.4263), 'val': tensor(2.4907)}
{'train': tensor(2.4211), 'val': tensor(2.4846)}
{'train': tensor(2.4210), 'val': tensor(2.4873)}
{'train': tensor(2.4208), 'val': tensor(2.4917)}
{'train': tensor(2.4163), 'val': tensor(2.4894)}
{'train': tensor(2.4164), 'val': tensor(2.4871)}
{'train': tensor(2.4154), 'val': tensor(2.4845)}
{'train': tensor(2.4175), 'val': tensor(2.4768)}
{'train': tensor(2.4

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(m.generate(context, max_new_tokens=12)[0].tolist())
genearated_tokens = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(genearated_tokens)

[0, 0, 39, 45, 44, 61, 58, 54, 75, 62, 71, 72, 9]

m wawasuse


Oz walfedecinowast ly Theded t. cashey. hecay dellindery wawntreeenneengowirelo tierediemawnde wave Fiely de wavoinffave decaly Q
Obedenly benssse hed,"

Oz tfowawawaveavored wa be b-hows. heceeders ts,"
OUThodeawave gl
Oz ulffs pansqunowas,"
Oz Oz hy re
Onodrwearawshere tushews,"

OUThangly. BL
O1 dirowawa wely b,"Yedelfave d wawb-he s,"Yede So tirin sl
m. chanoms t beche waranlino beelffly t trello thandisanom. wey we thedel. whawelfe
Oz enow t_ce s,"Thaino toowselfot ls,"Tharom. 


GPT: Decoder Only transformer

In [ ]:
class GPTLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()

    self.look_up_table = nn.Embedding(vocab_size, n_embed)
    self.positional_embedding = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)

    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.2)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)

  def forward(self, index, targets=None):

    B, T = index.shape
    logits = self.look_up_table(index)

    pos = self.positional_embedding(torch.arange(T, device=device))
    logits = logits + pos
    logits = self.blocks(logits)
    logits = self.ln_f(logits)
    logits = self.lm_head(logits)
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for token in range(max_new_tokens):
      logits, loss = self.forward(index)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)

      next_index = torch.multinomial(probs, num_samples=1, replacement=True)

      index = torch.cat((index, next_index), dim=-1)

    return index

model = GPTLanguageModel(len(chars))
m = model.to(device)

#index = torch.tensor([1, 2, 3, 4])

#logits = model(index, targets=index)
#print(logits.shape)

BLOCK

In [ ]:
class Block(nn.Module):
  def __init__(self, n_embed, n_head=n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(head_size, n_head)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)
    self.ffwd = FeedFoward(n_embed)

  def forward(self, x):
    y = self.sa(x)
    x = self.ln1(x + y)
    y = self.ffwd(x)
    x = self.ln2(x + y)

    return x


FEED FORWARD NEURAL NETWORK

In [ ]:
class FeedFoward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(n_embed, 4*n_embed), nn.ReLU(), nn.Linear(4*n_embed, n_embed), nn.Dropout(dropout))

  def forward(self, x):
    x = self.net(x)
    return x


MULTI HEAD ATTENTION

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, head_size, n_head):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
    self.proj = nn.Linear(n_head * head_size, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    x = torch.cat([head(x) for head in self.heads], dim=-1)
    x = self.proj(x)
    x = self.dropout(x)
    return x


HEAD

In [ ]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.querry = nn.Linear(n_embed, head_size)
    self.value = nn.Linear(n_embed, head_size)
    self.key = nn.Linear(n_embed, head_size)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    B, T, C = x.shape
    q = self.querry(x)
    k = self.querry(x)

    wei = q @ k.transpose(-2,-1) * k.shape[-1] ** - 0.5
    wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    v = self.value(x)

    out = wei @ v
    return out



DATA EXTRACT FILE

In [ ]:
import os
import lzma
from tqdm import tqdm

def xz_files_in_dir(directory):
  files = []
  for filename in os.listdir(directory):
    if filename.endswith('xz') and os.path.isfile(os.path.join(directory, filename)):
      files.append(filename)

  return files


folder_path = "/Users/redfique/Downloads/openwebtext"
output_file = 'output{}.txt'
vocab_file = 'vocab.txt'
split_files = int(input("How many files would you like to split this into?"))


files = xz_files_in_dir(folder_path)

total_files = len(files)

vocab = set()

max_count = total_files // split_files if split_files != 0 else total_files

for i in range(split_files):
  with open(output_file.format(i), 'w', encoding='utf-8') as outfile:
    for count, filename in enumerate(tqdm(files[:max_count], total=max_count)):
      if count > max_count:
        break

      file_path = os.path.join(folder_path, filename)
      with lzma.open(file_path, 'rt', encoding="utf-8") as infile:
        text = infile.read()
        outfile.write(text)
        characters = set(text)

        vocab.update(characters)

    files = files[max_count:]

with open(vocab_file, "w", encoding='utf-8') as vfile:
      for char in vocab:
        vfile.write(char)




TRAINING AND VAL SPLITS

In [ ]:
import os
import lzma
from tqdm import tqdm

def xz_files_in_dir(directory):
  files = []
  for filename in os.listdir(directory):
    if filename.endswith('xz') and os.path.isfile(os.path.join(directory, filename)):
      files.append(filename)

  return files


folder_path = "/Users/redfique/Downloads/openwebtext"
output_train_file = 'output_train.txt'
output_val_file = 'output_val.txt'
vocab_file = 'vocab.txt'

files = xz_files_in_dir(folder_path)

total_files = len(files)
split_index = int(total_files * 0.9)

train_files = files[:split_index]
val_files = files[split_index:]

vocab = set()

with open(output_train_file, 'w', encoding='utf-8') as outfile:
  for count, filename in enumerate(train_files, total=len(train_files)):

    file_path = os.path.join(folder_path, filename)
    with lzma.open(file_path, 'rt', encoding="utf-8") as infile:
      text = infile.read()
      outfile.write(text)
      characters = set(text)

      vocab.update(characters)

with open(output_val_file, 'w', encoding='utf-8') as outfile:
  for count, filename in enumerate(val_files, total=len(val_files)):

    file_path = os.path.join(folder_path, filename)
    with lzma.open(file_path, 'rt', encoding="utf-8") as infile:
      text = infile.read()
      outfile.write(text)
      characters = set(text)

      vocab.update(characters)

with open(vocab_file, "w", encoding='utf-8') as vfile:
      for char in vocab:
        vfile.write(char)




GET BATCH FOR OPENWEBTEXT

In [ ]:
import mmap
import random
def get_random_chunk(split):
  filename = "output_train.txt" if split == "train" else "output_val.txt"
  with open(filename, "rb") as f:
    with mmap.mmap(f.fileno(), 0, access=mmap.ACCES_READ) as mm:
      # file size and random positions
      file_size = len(mm)
      start_pos = random.randint(0, block_size*batch_size)

      # seek to the start position and read the block of text






VOCABULARIES OF THE OPENWEBTEXT

In [ ]:
with open('vocab.txt', 'r', encoding='utf-8') as f:
  text = f.read()
  chars = sorted(list(set(text)))

vocab_size = len(chars)


LOADING MODEL IF EXISTS

In [ ]:
print('loading model parameters .....')
with open('model_001.pkl', 'rb') as f:
  model = pickle.load(f)
model.to(device)
print('loaded successfully')

